In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import plotly.express as px
from datetime import datetime, timedelta
from tqdm.notebook import trange, tqdm

from postgres_data import session_scope, ProbeRequest, get_data
from sqlalchemy import func

In [21]:
import netaddr
import requests
import json

def get_mac_vendor(mac):
    URLs = ['http://macvendors.co/api/%s', 'http://www.macvendorlookup.com/api/v2/%s', 'https://api.macvendors.com/%s',]
    vendor = 'UNKNOWN'

    try:
        parsed_mac = netaddr.EUI(mac)
        vendor = parsed_mac.oui.registration().org
    except netaddr.core.NotRegisteredError as e:
        for url in URLs:
            r = requests.get(url % mac)
            if r and r.status_code == 200:
                try:
                    # print(r.json())
                    res = r.json()
                    if 'error' in res:
                        continue
                    if 'result' in res.keys() and 'company' in res['result'].keys():
                        vendor = res['result']['company']
                        break
                except json.JSONDecodeError:
                    # print(r.text)
                    if r.text:
                        vendor = r.text
                        break
    return vendor


In [3]:
# data = get_data()

In [11]:
with session_scope() as session:
    print(session.query(ProbeRequest.make).filter(func.lower(ProbeRequest.macaddress).startswith("da:a1:19")).distinct(ProbeRequest.macaddress).count())

26483


In [22]:
device = ""
print(get_mac_vendor(device))
with session_scope() as session:
    # print(session.query(ProbeRequest.make).filter(ProbeRequest.make == "UNKNOWN").distinct(ProbeRequest.macaddress).count())
    # print(session.query(ProbeRequest.make).filter(ProbeRequest.macaddress == "04:33:c2:67:c1:d1").count())
    device_probes = session.query(ProbeRequest).filter(func.lower(ProbeRequest.macaddress) == func.lower(device)).all()
    device_probes_df = pd.DataFrame([probe.to_dict() for probe in device_probes])
px.histogram(device_probes_df, x="date")

Apple, Inc.


In [23]:
device_probes_df.shape

(372, 5)

In [9]:
def get_probes():
    with session_scope() as s:
        return s.query(ProbeRequest)
Probe = get_probes

In [10]:
now = datetime.now()
before = datetime.now()-timedelta(hours=2)
Probe().filter(ProbeRequest.date.between(before, now)).count()

0

In [11]:
mac = data.macaddress.unique()[0]

In [12]:
data[data.macaddress==mac]

,id,date,macaddress,make,ssid,rssi
0,2276795,2020-11-16 17:05:17.839214,3a:ca:98:75:fe:b6,UNKNOWN,,-59


In [13]:


print(mac_make)

NameError: name 'mac_make' is not defined

In [8]:
mac_data = data[data.macaddress == mac]

print(mac_data.date.min(), mac_data.date.max())

2020-10-30 00:07:44.965614 2020-11-14 11:49:56.336838


In [9]:
mac_data.ssid.unique()

array(['WLAN-HDLBVC', 'eduroam', '', 'WLAN-2DDB06'], dtype=object)

In [10]:
# fig = px.timeline(mac_data, x='date', y='rssi')
# fig.show()

In [11]:
len(one_hr_mac_indexes)

NameError: name 'one_hr_mac_indexes' is not defined

In [66]:
data

,id,date,macaddress,make,ssid,rssi
0,2248362,2020-11-10 20:29:00.408120,ec:9b:f3:82:eb:19,SAMSUNG ELECTRO-MECHANICS(THAILAND),,-61
1,2248361,2020-11-10 20:28:44.907968,ec:9b:f3:82:eb:19,SAMSUNG ELECTRO-MECHANICS(THAILAND),,-63
2,2248360,2020-11-10 20:28:07.976759,ec:9b:f3:82:eb:19,SAMSUNG ELECTRO-MECHANICS(THAILAND),,-67
4,2248358,2020-11-10 20:26:18.732897,84:cf:bf:8e:cb:5e,Fairphone,,-70
11,2248351,2020-11-10 20:22:13.209204,40:b8:9a:88:3a:47,"Hon Hai Precision Ind. Co.,Ltd.",,-70
...,...,...,...,...,...,...
99986,2148376,2020-10-18 11:06:05.087214,00:04:4b:56:fd:99,NVIDIA,FRITZ!Box 7490,-57
99987,2148375,2020-10-18 11:05:08.525163,00:04:4b:56:fd:99,NVIDIA,FRITZ!Box 7490,-60
99995,2148367,2020-10-18 11:01:55.873136,00:04:4b:56:fd:99,NVIDIA,FRITZ!Box 7490,-60
99996,2148366,2020-10-18 11:01:44.744558,00:04:4b:56:fd:99,NVIDIA,FRITZ!Box 7490,-59


In [70]:
for hour in range(12):
    one_hr = data[(data.date > datetime.fromisoformat(f'2020-11-09 {hour:02}:00:00.000')) & (data.date < datetime.fromisoformat(f'2020-11-09 {hour:02}:00:00.000'))]
    print(one_hr)
    one_hr_mac_indexes = {mac: i for i, mac in enumerate(one_hr.macaddress.unique())}
    print(one_hr_mac_indexes)
    arr = np.zeros([len(one_hr_mac_indexes),60])
    for i in range(0,60):
        start_date = datetime.fromisoformat(f'2020-11-09 {hour:02}:{i:02}:00.000')
        if i < 59:
            end_date = datetime.fromisoformat(f'2020-11-09 {hour:02}:{i+1:02}:00.000')
        else:
            datetime.fromisoformat(f'2020-11-09 {hour:02}:00:00.000')
        
        min_data = one_hr[(one_hr.date > start_date) & (one_hr.date < end_date)]

        for mac in min_data.macaddress.unique():
            arr[one_hr_mac_indexes[mac]][i] = min_data[min_data.macaddress == mac].shape[0]
    print(arr)
    fig = px.imshow(arr, color_continuous_scale=px.colors.sequential.BuGn_r)
    fig.update_traces(hovertemplate="date={hour:02}:%{x:02} <br> mac=%{y} <br> bursts=%{z}")
    fig.show()
    if hour > 3:
        break

Empty DataFrame
Columns: [id, date, macaddress, make, ssid, rssi]
Index: []
{}
[]


Empty DataFrame
Columns: [id, date, macaddress, make, ssid, rssi]
Index: []
{}
[]


Empty DataFrame
Columns: [id, date, macaddress, make, ssid, rssi]
Index: []
{}
[]


Empty DataFrame
Columns: [id, date, macaddress, make, ssid, rssi]
Index: []
{}
[]


Empty DataFrame
Columns: [id, date, macaddress, make, ssid, rssi]
Index: []
{}
[]


In [48]:
cnts = data.macaddress.value_counts()
cnts = cnts[cnts > 2]
px.bar(cnts, log_y=True, width=1500)

In [89]:
mac_most = cnts.index.to_list()[10:15]

In [90]:
most_macs=data[data.macaddress.isin(mac_most)]

In [91]:
for mac in most_macs.macaddress.unique():
    print(mac, get_mac_vendor(mac))
    print(most_macs[most_macs.macaddress==mac].ssid.unique())
    fig=px.line(most_macs[most_macs.macaddress==mac], x='date', y='rssi', width=1200)
    fig.show()
    

40:b8:9a:88:3a:47 Hon Hai Precision Ind. Co.,Ltd.
['' 'alice_im_w-land']


84:cf:bf:8e:cb:5e Fairphone
['']


ec:08:6b:08:15:17 TP-LINK TECHNOLOGIES CO.,LTD.
['' 'FRITZ!Box 6591 Cable PF' 'FRITZ!Box 7490']


bc:dd:c2:fd:e3:35 Espressif Inc.
['FRITZ!Box 6591 Cable PF' '022696']


2c:4d:79:52:ad:ac GoerTek Inc.
['' 'FRITZ!Box 7490']
